In [3]:
!pip install catboost lightgbm

In [121]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
test = pd.read_csv("test.csv")
df = pd.read_csv("train.csv")
df_new = df.drop("target", axis = 1)

In [122]:
df

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,target,Id
0,8,14,4,1334.0,1440.0,XE,430,N13553,54.0,FAT,SAN,314,3.0,9.0,0,FYNC08ZRT1MS
1,5,3,6,1706.0,2016.0,XE,2567,N12924,103.0,BNA,EWR,748,9.0,18.0,1,4M3EPJ0BFKNV
2,7,11,5,1323.0,1507.0,XE,2989,N19966,125.0,CLT,IAH,913,6.0,33.0,1,OR30D5LJS4AF
3,12,11,4,2039.0,2238.0,WN,3743,N215WN,224.0,BWI,DEN,1491,5.0,10.0,1,0PRTFL3QD5ZI
4,5,23,5,1628.0,1744.0,UA,926,N347UA,60.0,LAX,SFO,337,3.0,13.0,1,DR7UZO31NC2A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,11,22,6,1119.0,1238.0,AA,2421,N383AA,174.0,DFW,LAX,1235,10.0,15.0,0,KHV6OJ08LE0S
1999996,2,25,1,759.0,858.0,AA,1109,N431AA,36.0,DFW,AUS,190,6.0,17.0,0,WM56NFH3B82U
1999997,9,23,2,1226.0,2109.0,DL,162,N3763D,293.0,SEA,JFK,2421,30.0,20.0,0,K6UIWFZ0R1T7
1999998,7,26,6,729.0,929.0,US,1099,N956UW,97.0,PHL,JAX,742,3.0,20.0,0,RMFGZOY59UCK


In [123]:
columns = ["Month", "DayofMonth", "DayOfWeek", "UniqueCarrier", "Origin", "Dest"]

In [124]:
CatBoost = CatBoostClassifier(cat_features = columns, depth = 8)

In [125]:
target = df["target"]

In [126]:
df_new = df_new.fillna(0)

In [127]:
df_new['DepTime'] = df_new['DepTime'] / 100

In [128]:
df_new['ArrTime'] = df_new['ArrTime'] / 100

In [129]:
origin_mte = df[['Origin', 'target']].groupby('Origin')['target'].mean()
origin_mte.name = 'Origin_MTE'
df_new = pd.merge(df_new, origin_mte, how='left', on='Origin')

In [130]:
Dest_mte = df[['Dest', 'target']].groupby('Dest')['target'].mean()
Dest_mte.name = 'Dest_MTE'
df_new = pd.merge(df_new, Dest_mte, how='left', on='Dest')

In [131]:
df_new = df_new.drop("Id", axis = 1)
df_new = df_new.drop("TailNum", axis = 1)

In [132]:
smth = df['TaxiIn'] + df['TaxiOut'] +  df['AirTime']
df.insert(loc=len(df.columns), column='Smth', value=smth) 

In [133]:
df_new.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Origin_MTE,Dest_MTE
0,8,14,4,13.34,14.40,XE,430,54.0,FAT,SAN,314,3.0,9.0,0.203920,0.301685
1,5,3,6,17.06,20.16,XE,2567,103.0,BNA,EWR,748,9.0,18.0,0.302108,0.418648
2,7,11,5,13.23,15.07,XE,2989,125.0,CLT,IAH,913,6.0,33.0,0.297510,0.250599
3,12,11,4,20.39,22.38,WN,3743,224.0,BWI,DEN,1491,5.0,10.0,0.314572,0.277956
4,5,23,5,16.28,17.44,UA,926,60.0,LAX,SFO,337,3.0,13.0,0.289365,0.344834


In [134]:
Cat = CatBoost.fit(df_new, target)

Learning rate set to 0.264529
0:	learn: 0.6218822	total: 3.38s	remaining: 56m 18s
1:	learn: 0.5790288	total: 5.93s	remaining: 49m 21s
2:	learn: 0.5547740	total: 7.99s	remaining: 44m 14s
3:	learn: 0.5441960	total: 10.6s	remaining: 44m 2s
4:	learn: 0.5350155	total: 13.6s	remaining: 45m
5:	learn: 0.5267020	total: 16.4s	remaining: 45m 8s
6:	learn: 0.5232482	total: 18.4s	remaining: 43m 36s
7:	learn: 0.5207328	total: 20.6s	remaining: 42m 37s
8:	learn: 0.5173935	total: 23.2s	remaining: 42m 36s
9:	learn: 0.5144051	total: 25.3s	remaining: 41m 49s
10:	learn: 0.5127018	total: 27.6s	remaining: 41m 17s
11:	learn: 0.5085873	total: 30.1s	remaining: 41m 15s
12:	learn: 0.5066499	total: 32.7s	remaining: 41m 19s
13:	learn: 0.5051288	total: 35.1s	remaining: 41m 12s
14:	learn: 0.5030209	total: 37.9s	remaining: 41m 27s
15:	learn: 0.5016603	total: 39.5s	remaining: 40m 32s
16:	learn: 0.5008519	total: 42s	remaining: 40m 27s
17:	learn: 0.4999964	total: 44.2s	remaining: 40m 10s
18:	learn: 0.4993348	total: 46.6s	

152:	learn: 0.4440348	total: 6m 25s	remaining: 35m 36s
153:	learn: 0.4433520	total: 6m 28s	remaining: 35m 32s
154:	learn: 0.4424951	total: 6m 30s	remaining: 35m 31s
155:	learn: 0.4417163	total: 6m 33s	remaining: 35m 27s
156:	learn: 0.4416362	total: 6m 35s	remaining: 35m 25s
157:	learn: 0.4407697	total: 6m 37s	remaining: 35m 20s
158:	learn: 0.4404357	total: 6m 40s	remaining: 35m 18s
159:	learn: 0.4403471	total: 6m 43s	remaining: 35m 16s
160:	learn: 0.4398796	total: 6m 45s	remaining: 35m 13s
161:	learn: 0.4397805	total: 6m 48s	remaining: 35m 10s
162:	learn: 0.4397156	total: 6m 50s	remaining: 35m 7s
163:	learn: 0.4391848	total: 6m 53s	remaining: 35m 8s
164:	learn: 0.4384516	total: 6m 56s	remaining: 35m 5s
165:	learn: 0.4383625	total: 6m 58s	remaining: 35m 2s
166:	learn: 0.4380744	total: 7m	remaining: 34m 59s
167:	learn: 0.4371847	total: 7m 3s	remaining: 34m 55s
168:	learn: 0.4362730	total: 7m 5s	remaining: 34m 53s
169:	learn: 0.4358513	total: 7m 8s	remaining: 34m 51s
170:	learn: 0.4357727

302:	learn: 0.4008243	total: 12m 48s	remaining: 29m 28s
303:	learn: 0.4007427	total: 12m 51s	remaining: 29m 27s
304:	learn: 0.3998082	total: 12m 54s	remaining: 29m 24s
305:	learn: 0.3997643	total: 12m 56s	remaining: 29m 21s
306:	learn: 0.3995494	total: 12m 59s	remaining: 29m 18s
307:	learn: 0.3995335	total: 13m 1s	remaining: 29m 16s
308:	learn: 0.3994842	total: 13m 3s	remaining: 29m 13s
309:	learn: 0.3993054	total: 13m 6s	remaining: 29m 9s
310:	learn: 0.3992685	total: 13m 8s	remaining: 29m 7s
311:	learn: 0.3992228	total: 13m 11s	remaining: 29m 4s
312:	learn: 0.3991329	total: 13m 13s	remaining: 29m 1s
313:	learn: 0.3987935	total: 13m 16s	remaining: 28m 59s
314:	learn: 0.3987103	total: 13m 18s	remaining: 28m 57s
315:	learn: 0.3986714	total: 13m 21s	remaining: 28m 54s
316:	learn: 0.3985852	total: 13m 24s	remaining: 28m 52s
317:	learn: 0.3983813	total: 13m 26s	remaining: 28m 49s
318:	learn: 0.3983341	total: 13m 29s	remaining: 28m 48s
319:	learn: 0.3982952	total: 13m 32s	remaining: 28m 45s


450:	learn: 0.3803369	total: 19m 26s	remaining: 23m 39s
451:	learn: 0.3803124	total: 19m 29s	remaining: 23m 37s
452:	learn: 0.3802729	total: 19m 32s	remaining: 23m 35s
453:	learn: 0.3800694	total: 19m 35s	remaining: 23m 33s
454:	learn: 0.3800120	total: 19m 37s	remaining: 23m 30s
455:	learn: 0.3799645	total: 19m 40s	remaining: 23m 28s
456:	learn: 0.3799510	total: 19m 42s	remaining: 23m 25s
457:	learn: 0.3799381	total: 19m 44s	remaining: 23m 22s
458:	learn: 0.3799235	total: 19m 47s	remaining: 23m 19s
459:	learn: 0.3793444	total: 19m 50s	remaining: 23m 17s
460:	learn: 0.3792990	total: 19m 52s	remaining: 23m 14s
461:	learn: 0.3792661	total: 19m 55s	remaining: 23m 12s
462:	learn: 0.3792580	total: 19m 57s	remaining: 23m 9s
463:	learn: 0.3792398	total: 20m	remaining: 23m 7s
464:	learn: 0.3788567	total: 20m 3s	remaining: 23m 4s
465:	learn: 0.3788104	total: 20m 6s	remaining: 23m 2s
466:	learn: 0.3781319	total: 20m 9s	remaining: 23m
467:	learn: 0.3780922	total: 20m 11s	remaining: 22m 57s
468:	le

598:	learn: 0.3652235	total: 26m 6s	remaining: 17m 28s
599:	learn: 0.3652009	total: 26m 9s	remaining: 17m 26s
600:	learn: 0.3651665	total: 26m 12s	remaining: 17m 23s
601:	learn: 0.3651424	total: 26m 14s	remaining: 17m 21s
602:	learn: 0.3650467	total: 26m 17s	remaining: 17m 18s
603:	learn: 0.3650288	total: 26m 20s	remaining: 17m 16s
604:	learn: 0.3649767	total: 26m 23s	remaining: 17m 13s
605:	learn: 0.3648188	total: 26m 25s	remaining: 17m 11s
606:	learn: 0.3647926	total: 26m 28s	remaining: 17m 8s
607:	learn: 0.3647519	total: 26m 30s	remaining: 17m 5s
608:	learn: 0.3647270	total: 26m 33s	remaining: 17m 2s
609:	learn: 0.3647063	total: 26m 35s	remaining: 17m
610:	learn: 0.3646866	total: 26m 38s	remaining: 16m 57s
611:	learn: 0.3646736	total: 26m 41s	remaining: 16m 55s
612:	learn: 0.3646375	total: 26m 44s	remaining: 16m 52s
613:	learn: 0.3645633	total: 26m 46s	remaining: 16m 50s
614:	learn: 0.3645339	total: 26m 48s	remaining: 16m 47s
615:	learn: 0.3645005	total: 26m 51s	remaining: 16m 44s
6

746:	learn: 0.3518821	total: 32m 22s	remaining: 10m 57s
747:	learn: 0.3517226	total: 32m 24s	remaining: 10m 55s
748:	learn: 0.3516924	total: 32m 27s	remaining: 10m 52s
749:	learn: 0.3516700	total: 32m 29s	remaining: 10m 49s
750:	learn: 0.3516398	total: 32m 32s	remaining: 10m 47s
751:	learn: 0.3516087	total: 32m 34s	remaining: 10m 44s
752:	learn: 0.3515999	total: 32m 37s	remaining: 10m 41s
753:	learn: 0.3513279	total: 32m 39s	remaining: 10m 39s
754:	learn: 0.3512926	total: 32m 42s	remaining: 10m 36s
755:	learn: 0.3511909	total: 32m 44s	remaining: 10m 34s
756:	learn: 0.3510209	total: 32m 46s	remaining: 10m 31s
757:	learn: 0.3510044	total: 32m 49s	remaining: 10m 28s
758:	learn: 0.3509717	total: 32m 51s	remaining: 10m 26s
759:	learn: 0.3509509	total: 32m 54s	remaining: 10m 23s
760:	learn: 0.3509398	total: 32m 56s	remaining: 10m 20s
761:	learn: 0.3509229	total: 32m 59s	remaining: 10m 18s
762:	learn: 0.3506907	total: 33m 1s	remaining: 10m 15s
763:	learn: 0.3505830	total: 33m 3s	remaining: 10

896:	learn: 0.3405052	total: 38m 41s	remaining: 4m 26s
897:	learn: 0.3404868	total: 38m 43s	remaining: 4m 23s
898:	learn: 0.3404683	total: 38m 45s	remaining: 4m 21s
899:	learn: 0.3404572	total: 38m 48s	remaining: 4m 18s
900:	learn: 0.3404486	total: 38m 51s	remaining: 4m 16s
901:	learn: 0.3403528	total: 38m 53s	remaining: 4m 13s
902:	learn: 0.3403383	total: 38m 56s	remaining: 4m 10s
903:	learn: 0.3403261	total: 38m 58s	remaining: 4m 8s
904:	learn: 0.3403019	total: 39m 1s	remaining: 4m 5s
905:	learn: 0.3402724	total: 39m 4s	remaining: 4m 3s
906:	learn: 0.3400876	total: 39m 6s	remaining: 4m
907:	learn: 0.3400745	total: 39m 8s	remaining: 3m 57s
908:	learn: 0.3397943	total: 39m 11s	remaining: 3m 55s
909:	learn: 0.3397275	total: 39m 14s	remaining: 3m 52s
910:	learn: 0.3396241	total: 39m 16s	remaining: 3m 50s
911:	learn: 0.3395438	total: 39m 19s	remaining: 3m 47s
912:	learn: 0.3395088	total: 39m 21s	remaining: 3m 45s
913:	learn: 0.3394748	total: 39m 23s	remaining: 3m 42s
914:	learn: 0.3394332

In [135]:
test = test.fillna(0)

In [136]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,AirTime,Origin,Dest,Distance,TaxiIn,TaxiOut,Id
0,8,15,5,806.0,957.0,UA,1157,N834UA,111.0,DFW,DEN,641,8.0,52.0,QKXNUJO50EPF
1,10,14,2,2020.0,2133.0,WN,1965,N203WN,49.0,PHX,ONT,325,4.0,20.0,Y2XFDOHQK6VM
2,6,1,7,2245.0,9.0,FL,910,N943AT,67.0,ATL,PHF,508,6.0,11.0,TU5QMACJHR10
3,12,31,3,1638.0,1753.0,OO,5732,N567SW,66.0,GEG,SEA,224,3.0,6.0,0731ET5Z9FWJ
4,10,23,4,1544.0,2358.0,AA,22,N329AA,289.0,LAX,JFK,2475,12.0,13.0,5BNGICLQFKSO


In [137]:
test = test.drop("TailNum", axis = 1)

In [138]:
test = test.fillna(0)

In [139]:
test['DepTime'] = test['DepTime'] / 100

In [140]:
test['ArrTime'] = test['ArrTime'] / 100

In [141]:
test = test.drop("Id", axis = 1)

In [142]:
df["Smth2"] = df['TaxiIn'] + df['TaxiOut'] +  df['AirTime']

In [143]:
test["Origin_MTE"] = df_new["Origin_MTE"]
test["Dest_MTE"] = df_new["Dest_MTE"]

In [144]:
proba = CatBoost.predict_proba(test)[:, 1]

In [145]:
submission = pd.read_csv("sample_submission.csv")

In [146]:
submission['target'] = proba

In [147]:
submission.to_csv("submission", index = False)